In [1]:
!pip install sentence-transformers

     |████████████████████████████████| 61kB 3.4MB/s 
     |████████████████████████████████| 645kB 9.5MB/s 
     |████████████████████████████████| 1.1MB 16.7MB/s 
     |████████████████████████████████| 3.8MB 32.8MB/s 
     |████████████████████████████████| 890kB 38.7MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.6.1-cp36-none-any.whl size=74031 sha256=0e28d7f3c4c7350be488d090763dd7489b6d654c25707ae7a1c841da5de9486b
  Stored in directory: /root/.cache/pip/wheels/d7/fa/17/2b081a8cd8b0a86753fb0e9826b3cc19f0207062c0b2da7008
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=e1f98cd323627c34007ecf2aa63c833b1529c3566d3c893769a4e771e1253841
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [0]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.io import imread, imsave
from skimage.transform import rescale
from skimage import img_as_ubyte
import pickle
import sentence_transformers

In [3]:
#4/zQFDcqJxIv-ozKrojm6LD8PrmNVQi31nQcpfDqU51i3XWOlyL9wh1ZU
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
import os
os.chdir("/content/drive/My Drive/text-to-image")
os.listdir()

['model',
 'Flickr8k_Dataset',
 'data',
 'Flickr8k_text',
 'src-juan',
 'src-paola',
 'src-ely',
 'resultados_pasados',
 'pickles',
 'logs',
 'ToDo.gdoc']

In [5]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# GLOBALS
TOKEN_DIR = "./Flickr8k_text/"

def decontracted(phrase):
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase 


# Función auxiliar para remover stopwords y otras normalizaciones de texto
def clean_sentences(dataframe ):
    table = str.maketrans('','',string.punctuation)
    SYNS = []
    for sentence in dataframe:
        sentence = decontracted(sentence)
        SYNS.append(sentence.strip())
    return SYNS

# Load data
dir_token = TOKEN_DIR + "Flickr8k.token.txt"
df = pd.read_csv(dir_token, sep='\t', names=['ID', 'Text'], index_col=False)
print(df.shape)

# Remueve entries with nan values
df.dropna(inplace=True)
df.isnull().values.any()
df

X = df.Text
y = df.ID

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)


(40460, 2)


In [0]:
# Remueve stopwords
synops_train = clean_sentences(X_train )
synops_test  = clean_sentences(X_test )

In [7]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')
test_embeddings = model.encode(synops_test)
train_embeddings = model.encode(synops_train)

100%|██████████| 405M/405M [00:08<00:00, 48.8MB/s]


In [0]:
from pickle import dump, load

with open("./pickles/embeddings_BERT_noprepro/E_train.pickle","wb") as f:
  pickle.dump(train_embeddings, f)

with open("./pickles/embeddings_BERT_noprepro/E_test.pickle","wb") as f:
  pickle.dump(test_embeddings, f)

In [0]:
len(test_embeddings[0])

768

In [0]:
len(train_embeddings)

32368